# A Next Product To Buy (NPTB) model using data which is hidden (for academic purposes only)
Next-Product-To-Buy modeling.

Importing relevant packages and data

In [24]:
import os

import numpy as np
import pandas as pd
import pyrsm as rsm
import statsmodels.formula.api as smf
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.families.links import logit

In [25]:
# worthwhile to check the working directory used
os.getcwd()

'/mnt/c/Users/Aditi Jain/Desktop/rsm-mgta455-bbb-nptb'

In [26]:
# loading data
bbb_nptb = pd.read_pickle("data/bbb_nptb.pkl")

Running a logistic regression using information about the offer we sent.

In [27]:
bbb_nptb["buyer_yes"] = (bbb_nptb["buyer"] == "yes").astype(int)
lr = smf.glm(
    formula="buyer_yes ~ offer + gender + last + total + child + \
             youth + cook + do_it + reference + art + geog",
    family=Binomial(link=logit()),
    data=bbb_nptb,
).fit()
# lr.summary()
# rsm.model_fit(lr)
rsm.or_ci(lr)

,index,OR,OR%,2.5%,97.5%,p.values,
1,offer[T.DIY],1.546,54.6%,1.407,1.700,< .001,***
2,offer[T.Cook],1.239,23.9%,1.124,1.366,< .001,***
3,gender[T.M],0.701,-29.9%,0.646,0.761,< .001,***
4,last,0.917,-8.3%,0.912,0.923,< .001,***
5,total,1.001,0.1%,1.000,1.001,< .001,***
6,child,0.978,-2.2%,0.944,1.013,0.207,
7,youth,1.146,14.6%,1.089,1.206,< .001,***
8,cook,1.223,22.3%,1.185,1.261,< .001,***
9,do_it,1.104,10.4%,1.053,1.157,< .001,***
10,reference,1.264,26.4%,1.195,1.336,< .001,***


We want to predict what each customer **would have done** if we had sent
him any one of the three offers.

### Creating predictions

We want to predict what each person would have done if they had been
sent, for example, the _Art_ offer. We can modify the prediction data by
setting the value of the "offer" column and (re) running the model prediction for each book type. This is what is being done now.

In [28]:
bbb_nptb["p_art"] = lr.predict(bbb_nptb.assign(offer="Art"))
bbb_nptb["p_diy"] = lr.predict(bbb_nptb.assign(offer="DIY"))
bbb_nptb["p_cook"] = lr.predict(bbb_nptb.assign(offer="Cook"))

Which offer to extend?

Using `idxmax` function to automatically find the best offer for
each customer

In [29]:
bbb_nptb["to_offer"] = (
    bbb_nptb[["p_art", "p_diy", "p_cook"]]
    .idxmax(axis=1)
    .str.replace("p_art", "Art")
    .replace("p_diy", "DIY")
    .replace("p_cook", "Cook")
)
#data check
bbb_nptb

,acctnum,offer,buyer,gender,state,zip,zip3,first,last,book,...,cook,do_it,reference,art,geog,buyer_yes,p_art,p_diy,p_cook,to_offer
0,10001,Art,no,M,MD,21229,212,37,35,29,...,1,0,0,0,1,0,0.006855,0.010562,0.008480,DIY
1,10002,Art,no,M,PA,18212,182,29,13,45,...,0,0,0,1,0,0,0.061482,0.091988,0.075074,DIY
2,10003,Cook,no,F,NJ,07029,070,23,15,27,...,1,0,1,0,0,0,0.051485,0.077440,0.063016,DIY
3,10004,Cook,no,M,NY,11354,113,15,9,29,...,1,0,0,0,1,0,0.058402,0.087522,0.071365,DIY
4,10005,DIY,no,M,NY,11733,117,9,7,27,...,1,0,1,0,0,0,0.076909,0.114138,0.093572,DIY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,49996,NaN,NaN,M,PA,19335,193,47,21,90,...,1,0,1,0,2,0,0.042615,0.064402,0.052269,DIY
39996,49997,NaN,NaN,F,MD,20653,206,11,9,26,...,0,1,0,0,0,0,0.078370,0.116217,0.095317,DIY
39997,49998,NaN,NaN,M,PA,19087,190,19,13,29,...,1,0,0,0,1,0,0.044562,0.067274,0.054632,DIY
39998,49999,NaN,NaN,M,NJ,08648,086,25,5,100,...,1,1,0,3,0,0,0.206029,0.286372,0.243295,DIY


This command provides a label for the category with the maximum predicted
probability of buying (i.e., "p_art", "p_diy", "p_cook"). Lets use this
option and also create a variable `p_target` that captures the probability
of responding for the best offer selected for a customer. We use `max` with
`axis=1` here  because we want the maximum value for each customer (i.e.,
each row in the data)

In [30]:
bbb_nptb["p_target"] = bbb_nptb[["p_art", "p_diy", "p_cook"]].max(axis=1)
bbb_nptb

,acctnum,offer,buyer,gender,state,zip,zip3,first,last,book,...,do_it,reference,art,geog,buyer_yes,p_art,p_diy,p_cook,to_offer,p_target
0,10001,Art,no,M,MD,21229,212,37,35,29,...,0,0,0,1,0,0.006855,0.010562,0.008480,DIY,0.010562
1,10002,Art,no,M,PA,18212,182,29,13,45,...,0,0,1,0,0,0.061482,0.091988,0.075074,DIY,0.091988
2,10003,Cook,no,F,NJ,07029,070,23,15,27,...,0,1,0,0,0,0.051485,0.077440,0.063016,DIY,0.077440
3,10004,Cook,no,M,NY,11354,113,15,9,29,...,0,0,0,1,0,0.058402,0.087522,0.071365,DIY,0.087522
4,10005,DIY,no,M,NY,11733,117,9,7,27,...,0,1,0,0,0,0.076909,0.114138,0.093572,DIY,0.114138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,49996,NaN,NaN,M,PA,19335,193,47,21,90,...,0,1,0,2,0,0.042615,0.064402,0.052269,DIY,0.064402
39996,49997,NaN,NaN,F,MD,20653,206,11,9,26,...,1,0,0,0,0,0.078370,0.116217,0.095317,DIY,0.116217
39997,49998,NaN,NaN,M,PA,19087,190,19,13,29,...,0,0,0,1,0,0.044562,0.067274,0.054632,DIY,0.067274
39998,49999,NaN,NaN,M,NJ,08648,086,25,5,100,...,1,0,3,0,0,0.206029,0.286372,0.243295,DIY,0.286372


Creating a pivot table to see which books should be offered to
the customers

In [31]:
pd.crosstab(index=bbb_nptb["to_offer"], columns="count").apply(rsm.format_nr)

col_0,count
to_offer,
DIY,"40,000"


The problem here is that the data is not customized. The model predicts that every customer should be sent the DIY book.

The whole point of customization is that different offers may work better for different customers. In other words, we want to customize offers because we think that there might be an interaction between (1) who the customer is and (2) how effective the offer is. The model output is created below:

In [32]:
lri = smf.glm(
    formula="buyer_yes ~ offer + gender + last + total + child + \
             youth + cook + do_it + reference + art + geog + \
             offer:gender + offer:last + offer:total + offer:child + \
             offer:youth + offer:cook + offer:do_it + offer:reference + \
             offer:art + offer:geog",
    family=Binomial(link=logit()),
    data=bbb_nptb,
).fit()
rsm.or_ci(lri)

,index,OR,OR%,2.5%,97.5%,p.values,
1,offer[T.DIY],3.164,216.4%,2.314,4.327,< .001,***
2,offer[T.Cook],0.045,-95.5%,0.031,0.065,< .001,***
3,gender[T.M],0.471,-52.9%,0.403,0.551,< .001,***
4,offer[T.DIY]:gender[T.M],0.730,-27.0%,0.588,0.905,0.004,**
5,offer[T.Cook]:gender[T.M],5.859,485.9%,4.547,7.549,< .001,***
6,last,0.901,-9.9%,0.890,0.913,< .001,***
7,offer[T.DIY]:last,0.916,-8.4%,0.899,0.934,< .001,***
8,offer[T.Cook]:last,1.055,5.5%,1.038,1.073,< .001,***
9,total,1.001,0.1%,1.000,1.002,0.039,*
10,offer[T.DIY]:total,1.000,-0.0%,0.999,1.001,0.829,


Now using the results from the
new, more flexible, model. Providing names
`p_arti`, `p_diyi`, and `p_cooki` to store the predictions from the model
with interactions

In [33]:
bbb_nptb["p_arti"] = lri.predict(bbb_nptb.assign(offer="Art"))
bbb_nptb["p_diyi"] = lri.predict(bbb_nptb.assign(offer="DIY"))
bbb_nptb["p_cooki"] = lri.predict(bbb_nptb.assign(offer="Cook"))

In [34]:
bbb_nptb["to_offeri"] = (
    bbb_nptb[["p_arti", "p_diyi", "p_cooki"]]
    .idxmax(axis=1)
    .str.replace("p_arti", "Art")
    .replace("p_diyi", "DIY")
    .replace("p_cooki", "Cook")
)

This command provides a label for the category with the maximum predicted
probability of buying (i.e., "p_arti", "p_diyi", "p_cooki"). Lets use this
option and also create a variable `p_targeti` that captures the probability
of responding for the best offer selected for a customer.

In [35]:
bbb_nptb["p_targeti"] = bbb_nptb[["p_arti", "p_diyi", "p_cooki"]].max(axis=1)

Pivot table to see which books should be offered


In [36]:
pd.crosstab(index=bbb_nptb["to_offeri"], columns="count").apply(rsm.format_nr)

col_0,count
to_offeri,
Art,"10,253"
Cook,"15,486"
DIY,"14,261"


Now lets create a table with the average purchase probabilities if we (1)
sent the Art book to everyone, or (2) sent the DIY book to everyone, or (3)
sent the Cook book to everyone, or (4) targeted the book that a customer is
most likely to buy according to our model with interactions

In [37]:
bbb_nptb[["p_arti", "p_diyi", "p_cooki", "p_targeti"]].agg("mean").sort_values(
    ascending=False
).apply(rsm.format_nr, perc=True)

p_targeti     22.5%
p_diyi        13.0%
p_cooki      10.83%
p_arti        8.99%
dtype: object

### Accounting for profitability

So far we have picked offers for each customer according to his/her predicted purchase probability. However, that is not the right criterion if offers differ in profitability. Given the following: The profit from selling the "Art History of Florence" is \\$6, the profit from selling "Paint Like a Pro" is \\$4, and the profit from selling "Vegetarian Cooking for Everyone" is \\$7.

Calculating the expected profit for each book and each customer.

In [38]:
bbb_nptb["ep_art"] = bbb_nptb["p_arti"] * 6
bbb_nptb["ep_diy"] = bbb_nptb["p_diyi"] * 4
bbb_nptb["ep_cook"] = bbb_nptb["p_cooki"] * 7

In [39]:
bbb_nptb["to_offer_ep"] = (
    bbb_nptb[["ep_art", "ep_diy", "ep_cook"]]
    .idxmax(axis=1)
    .str.replace("ep_art", "Art")
    .replace("ep_diy", "DIY")
    .replace("ep_cook", "Cook")
)

Finally, create a variable `ep_target` that captures the result from
targeting a customer with the book with the highest expected profit:

In [40]:
bbb_nptb["ep_target"] = bbb_nptb[["ep_art", "ep_diy", "ep_cook"]].max(axis=1)

Pivot table to see which book should be offered

In [41]:
pd.crosstab(index=bbb_nptb["to_offer_ep"], columns="count").applymap(rsm.format_nr)

col_0,count
to_offer_ep,
Art,"13,067"
Cook,"17,615"
DIY,"9,318"


Calculate average expected profits if we (1) sent the Art book to everyone,
or (2) sent the DIY book to everyone, or (3) sent the Cook book to everyone,
or (4) targeted using the book with the highest expected profit for each
individual customer

In [42]:
(
    bbb_nptb[["ep_art", "ep_diy", "ep_cook", "ep_target"]]
    .agg("mean")
    .sort_values(ascending=False)
    .apply(rsm.format_nr, sym="$")
)

ep_target    $1.26
ep_cook      $0.76
ep_art       $0.54
ep_diy       $0.52
dtype: object

The expected profit per customer from targeting is substantially higher, as we might expect. If we extrapolate this result to the remaining customers in the database, we expect the following profits:

In [43]:
profit_logit = bbb_nptb["ep_target"].agg("mean") * 520000
print(f"Expected profit from offer customization: ${profit_logit.round(2):,}")

Expected profit from offer customization: $654,514.7
